In [2]:
# Cell 1: Import necessary libraries
import numpy as np
import music21
from tensorflow.keras.models import load_model
from music21 import note, chord, stream

In [3]:
# Cell 2: Helper functions
def parse_midi_file(file_path):
    """Extract notes and chords from a MIDI file."""
    try:
        midi_stream = music21.converter.parse(file_path)
        notes = []
        for element in midi_stream.flat.notes:
            if isinstance(element, music21.note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, music21.chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))
        return notes
    except Exception as e:
        print(f"Error processing {file_path}: {e}")
        return []

def create_midi_file(predicted_notes, output_path):
    """Create a MIDI file from predicted notes."""
    offset = 0
    output_notes = []

    for note in predicted_notes:
        if '.' in note or note.isdigit():
            # Chord
            chord_notes = [music21.note.Note(int(n)) for n in note.split('.')]
            new_chord = music21.chord.Chord(chord_notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        else:
            # Note
            new_note = music21.note.Note(note)
            new_note.offset = offset
            output_notes.append(new_note)

        offset += 0.5  # Add a fixed step for note spacing

    midi_stream = music21.stream.Stream(output_notes)
    midi_stream.write('midi', fp=output_path)

def prepare_input_sequence(notes, note_to_int, sequence_length):
    """Prepare the input sequence for the model."""
    int_sequence = [note_to_int[note] for note in notes if note in note_to_int]
    pattern = int_sequence[-sequence_length:]
    input_sequence = np.reshape(pattern, (1, sequence_length, 1))
    input_sequence = input_sequence / float(len(note_to_int))
    return input_sequence

In [10]:
# Cell 3: Load the trained model and prepare data
model_path = 'best_lstm_music_model.keras'  # Path to the saved model
model = load_model(model_path)
print(f"Model loaded from {model_path}.")

input_midi_path = 'seed5.mid'  # Path to the input MIDI file
output_midi_path = 'generated_variation.mid'  # Path to save the generated MIDI variation

# Load and process the input MIDI file
sequence_length = 50  # Same sequence length as used in training
input_notes = parse_midi_file(input_midi_path)

if not input_notes:
    raise ValueError("No valid notes extracted from the input MIDI file.")

# Generate mapping from notes to integers and vice versa
unique_notes = sorted(set(input_notes))
note_to_int = {note: num for num, note in enumerate(unique_notes)}
int_to_note = {num: note for note, num in note_to_int.items()}

print(f"Unique notes in input: {len(unique_notes)}")

Model loaded from best_lstm_music_model.keras.
Unique notes in input: 33


In [11]:
def generate_variation(model, input_notes, note_to_int, int_to_note, sequence_length, num_notes):
    generated_notes = []
    
    input_notes = input_notes[-sequence_length:]
    input_notes_int = [note_to_int[note] for note in input_notes]
    input_sequence = np.array(input_notes_int).reshape((1, sequence_length, 1))
    
    for _ in range(num_notes):
        prediction = model.predict(input_sequence, verbose=0)
        predicted_index = np.argmax(prediction)
        
        # Handle index out of range
        if predicted_index >= len(int_to_note):
            predicted_index = predicted_index % len(int_to_note)
            
        predicted_note = int_to_note[predicted_index]
        
        try:
            if '.' in predicted_note:
                notes_in_chord = predicted_note.split('.')
                chord_notes = []
                for current_note in notes_in_chord:
                    try:
                        new_note = note.Note(int(current_note))
                        chord_notes.append(new_note)
                    except ValueError:
                        continue
                n = chord.Chord(chord_notes) if chord_notes else note.Note(60)
            else:
                n = note.Note(int(predicted_note)) if predicted_note.isdigit() else note.Note(60)
            
            generated_notes.append(n)
        except Exception:
            generated_notes.append(note.Note(60))
        
        predicted_index_reshaped = np.array([[predicted_index]]).reshape((1, 1, 1))
        input_sequence = np.append(input_sequence[:, 1:, :], predicted_index_reshaped, axis=1)
    
    score = stream.Score()
    part = stream.Part()
    for n in generated_notes:
        part.append(n)
    score.append(part)
    
    return score, generated_notes

# Usage:
num_notes_to_generate = 200
score, generated_notes = generate_variation(model, input_notes, note_to_int, int_to_note, sequence_length, num_notes_to_generate)

# Save the generated music
score.write('midi', fp='generated_music.mid')
print(f"Generated {len(generated_notes)} new notes.")

# Optionally show or play the generated music
score.show('midi')

Generated 200 new notes.
